<a href="https://colab.research.google.com/github/kawa-t/python_google_colaboratory/blob/main/Websc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#  ライブラリをインポート 
from bs4 import BeautifulSoup
import requests
url = "https://bitbank.cc/info/ripple/chart"
r = requests.get(url)
r.encoding = r.apparent_encoding
soup = BeautifulSoup(r.text, "lxml")
found = soup.find('h2', class_="currentprice")
print(found.text)

AttributeError: ignored